In [1]:
# macro
fileName = r'/home/liang.liang/code/Tools/python/pycharts/2019CW30.4.xlsx'
reportName = '20190725.html'
TargetCW = 30
TargetYear = 2019

In [2]:
from pyecharts import options as opts
from pyecharts.charts  import Page,Bar,Line,Grid
from pyecharts.charts import Liquid
import xlrd

In [3]:
data = xlrd.open_workbook(fileName)

table = data.sheets()[0]


Source =  {"KeyID":table.col_values(2,1),
         "funcGroup":table.col_values(4,1),
         "status":table.col_values(5,1),
         "group":table.col_values(6,1),
         "targetCW":table.col_values(7,1),
         "targetYear":table.col_values(8,1)
        }



Data =  {"KeyID":[],
         "funcGroup":[],
         "status":[],
         "group":[],
         "targetCW":[]
        }


TotalFeatures = len(Source["KeyID"])


In [4]:
for i in range(len(Source["KeyID"])):
    if (Source["targetCW"][i]!=''):
        if(int(Source["targetCW"][i]) <= TargetCW and int(Source["targetYear"][i]) == TargetYear):
            Data["KeyID"].append(Source["KeyID"][i])
            Data["funcGroup"].append(Source["funcGroup"][i])
            Data["status"].append(Source["status"][i])
            Data["group"].append(Source["group"][i])
            Data["targetCW"].append(Source["targetCW"][i])

In [5]:
# define calss for project management

In [6]:
class Features:
    """it's 37w project"""
    
    __group  = ("Audio","Media","Radio","Online","HMI","SDS","AndroidApp","AndroidOS"
                   ,"Update","Navi","System","CarConnect","DVR","PCC")    
    
    __status = ("New","In Analysis","Pending","In Implementation","In Testing","Released")
    
    __cw = [ ("CW%02d" %i)  for i in range(1,(TargetCW+1))]

    
    __featureLen = len(Data["KeyID"])
    
    __autowidth = 1200
    
    __countReleased = Data["status"].count("Released")
    
    
    def __init__(self,source,export):
        self.__source = source
        self.__export = export    
 
        self.__status_cw = {}
        for i in self.__status:
            self.__status_cw["%s" %i] = {c:0 for c in self.__cw }
            
        for s in self.__status:
            for c in self.__cw:
                temp_count = 0
                for i in range(len(Data["KeyID"])):
                    if(Data["status"][i] == s and Data["targetCW"][i] == c[2:] ):
                        temp_count +=1
                self.__status_cw[s][c] = temp_count
        

        self.__status_group_delay = {}
        for i in self.__status:
            self.__status_group_delay["%s" %i] = {j:0 for j in self.__group}


        for s in self.__status:
            for g in self.__group:
                temp_count = 0
                for ii in range(len(Data["KeyID"])):
                    if (Data["status"][ii] == s and Data["group"][ii] == g ):
                        temp_count +=1
        
                self.__status_group_delay[s][g] = temp_count
    
        
    def GetExportFileName(self):
        return self.__export
        
        
    def ProjectFinishRate(self):
#         liquid = Liquid(title="37W Project", title_text_size = 20
#               ,subtitle = "Features Completion Rate")
#         liquid.add("Liquid", [self.__countReleased/TotalFeatures],width = self.__autowidth/2)
        liquid = Liquid()
        liquid.add("Liquid", [self.__countReleased/TotalFeatures])
        liquid.set_global_opts(title_opts=opts.TitleOpts(title="Features Complete Rate"))
        
        return liquid


    

    
    def GroupStatus(self):
        bar1 = Bar()

        for i in self.__status:
            
            bar1.add_xaxis(self.__group)
            bar1.add_yaxis(str(i),list(self.__status_group_delay[i].values()),stack="stack1")
            bar1.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
            
            bar1.set_global_opts(title_opts=opts.TitleOpts(title="Group"),
                                 xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-45)),
                                 yaxis_opts=opts.AxisOpts
                                        (axislabel_opts=opts.LabelOpts(rotate=0),
                                         name = "",
                                         is_inverse = False,
                                         name_location = "end",
                                         name_gap = 5,
                                         name_rotate = 15,
                                         interval = 30,  # 无法在类目轴中使用
                                         split_number = 30,
                                         min_interval = 1,
                                         splitline_opts=opts.SplitLineOpts(is_show=True)
                                         ),
                                 #toolbox_opts=opts.ToolboxOpts(orient="vertical",pos_left = "90%")
            )

        return bar1

    
    
    
    def CWStatus(self):
        barCW = Bar()

        for i in self.__status:
            barCW.add_xaxis(self.__cw)
            barCW.add_yaxis(str(i),list(self.__status_cw[i].values()),stack="stack1")
            barCW.reversal_axis()
            barCW.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
     
            barCW.set_global_opts(
                title_opts=opts.TitleOpts(title="CW"),
                #xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-90)),
                #datazoom_opts=opts.DataZoomOpts()
                yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=0),
                                         name = "",
                                         is_inverse = False,
                                         name_location = "end",
                                         name_gap = 5,
                                         name_rotate = 15,
                                         interval = 30,  # 无法在类目轴中使用
                                         split_number = 30,
                                         min_interval = 1
                                        ),

            )

        return barCW


In [7]:
features = Features(fileName,reportName)

In [8]:
features.ProjectFinishRate().render_notebook()

In [9]:
features.GroupStatus().render_notebook()

In [10]:
features.CWStatus().render_notebook()

In [11]:
grid1 = Grid()
grid1.add(features.ProjectFinishRate(),grid_opts=opts.GridOpts(pos_top="10%"))

grid2 = Grid()
grid2.add(features.GroupStatus(), grid_opts=opts.GridOpts(pos_bottom="20%"))


grid3 = Grid()
grid3.add(features.CWStatus(), grid_opts=opts.GridOpts(pos_bottom="10%"))



page = Page(layout=Page.SimplePageLayout)

page.add(grid1)
page.add(grid2)
page.add(grid3)


page.render_notebook()
page.render(path = features.GetExportFileName())